In [1]:
import pandas

In [2]:
baseCsv = pandas.read_csv('./raw/crash_data.csv', sep=',', index_col=False, low_memory=False)

In [3]:
baseCsv

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,...,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location
0,MCP3170003V,240000438,Montgomery County Police,Property Damage Crash,01/03/2024 02:55:00 PM,NaN,NaN,NaN,NaN,IN FRONT OF 18900 BIRDSEYE DR,...,0,No,No,2017,LEXUS,SUV,NO MISUSE,39.165005,-77.249310,"(39.16500483, -77.24931)"
1,MCP3254003K,230072050,Montgomery County Police,Injury Crash,12/16/2023 12:36:00 PM,Maryland (State),GERMANTOWN RD,County,MIDDLEBROOK RD,NaN,...,35,No,No,2010,TOYT,PRIUS,NO MISUSE,39.178776,-77.267190,"(39.17877577, -77.26718974)"
2,EJ7887003Q,230074270,Gaithersburg Police Depar,Injury Crash,12/29/2023 12:00:00 PM,Maryland (State),GREAT SENECA HWY,Municipality,KENTLANDS BLVD,NaN,...,35,No,No,2021,SUBARU,FORRESTER,NO MISUSE,39.123574,-77.231769,"(39.12357374, -77.231769)"
3,MCP2674004J,230064598,Montgomery County Police,Property Damage Crash,11/05/2023 09:07:00 PM,Maryland (State),WOODFIELD RD,County,GLENDALOUGH RD,NaN,...,40,No,No,2019,DODGE,CHARGER,NO MISUSE,39.211742,-77.171461,"(39.21174219, -77.17146065)"
4,MCP25280008,230067019,Montgomery County Police,Property Damage Crash,11/18/2023 12:40:00 AM,Maryland (State),CLARKSBURG RD,Maryland (State),CLARKSBURG RD,NaN,...,20,No,No,2014,NISSAN,ROGUE,NaN,39.228915,-77.289091,"(39.22891483, -77.28909117)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172100,DM8338000C,16010228,Takoma Park Police Depart,Property Damage Crash,03/01/2016 10:01:00 AM,Municipality,NEW HAMPSHIRE SERVICE LA,Municipality,PRINCE GEORGES AVE,NaN,...,25,No,No,2016,FORD,EXPLORER,NO MISUSE,38.972560,-76.997466,"(38.97255976, -76.99746609)"
172101,MCP1182001S,170519976,Montgomery County Police,Property Damage Crash,07/19/2017 02:22:00 PM,County,CEDAR LA,Maryland (State),OLD GEORGETOWN RD,NaN,...,35,No,No,2008,NISSAN,ALTIMA,NO MISUSE,39.004640,-77.108502,"(39.00464, -77.10850167)"
172102,MCP1453008X,200046217,Montgomery County Police,Property Damage Crash,11/23/2020 07:37:00 AM,Maryland (State),RIDGE RD,Unknown,LITTLE SENECA PKWY,NaN,...,40,No,No,2008,CHEVROLET,EXPRESS,NO MISUSE,39.228963,-77.236757,"(39.22896333, -77.23675667)"
172103,MCP2568000M,190056701,Montgomery County Police,Property Damage Crash,11/23/2019 11:23:00 PM,County,SHADY GROVE RD,Ramp,RAMP 9 FR IS 370 EB TO SHADY GROVE RD,NaN,...,35,No,No,2018,FORD,TRANSIT VAN,NO MISUSE,39.120440,-77.180047,"(39.12043995, -77.18004738)"


In [5]:
baseCsv['Report Number'] = baseCsv['Report Number'].astype(str).str.strip()
baseCsv = baseCsv.drop_duplicates(subset=['Report Number'], keep='first').reset_index(drop=True)


In [7]:
baseCsv.isnull().sum()

Report Number                         0
Local Case Number                     0
Agency Name                           0
ACRS Report Type                      0
Crash Date/Time                       0
Route Type                        12702
Road Name                         12038
Cross-Street Type                 12721
Cross-Street Name                 12048
Off-Road Description              84818
Municipality                      86411
Related Non-Motorist              91566
Collision Type                      489
Weather                            7848
Surface Condition                 14338
Light                               800
Traffic Control                   16178
Driver Substance Abuse            17454
Non-Motorist Substance Abuse      92665
Person ID                             0
Driver At Fault                       0
Injury Severity                       0
Circumstance                      78729
Driver Distracted By                  0
Drivers License State              6891


In [6]:
# Group 1: Report and Case Information
df_report_case = baseCsv[['Report Number', 'Local Case Number', 'Agency Name', "ACRS Report Type", "Crash Date/Time", 'Municipality', 'Location']]
df_report_case.insert(0, 'ID', range(1, 1 + len(df_report_case)))

# Group 2: Road and Location Information 
df_road_location = baseCsv[['Route Type', 'Road Name', 'Cross-Street Type', 
                       'Cross-Street Name', 'Off-Road Description', 'Latitude', 'Longitude']]
df_road_location.insert(0, 'ID', range(1, 1 + len(df_road_location)))


# Group 3: Vehicle Information 
df_vehicle = baseCsv[['Vehicle ID', 'Vehicle Damage Extent', 
                 'Vehicle First Impact Location', 'Vehicle Second Impact Location', 
                 'Vehicle Body Type', 'Vehicle Movement', 'Vehicle Continuing Dir', 
                 'Vehicle Going Dir', 'Speed Limit', 'Driverless Vehicle', 
                 'Parked Vehicle', 'Vehicle Year', 'Vehicle Make', 'Vehicle Model', 
                 'Equipment Problems']]
df_vehicle.insert(0, 'ID', range(1, 1 + len(df_vehicle)))

# Group 4: Driver and Person Information
df_driver_person = baseCsv[['Person ID', 'Driver At Fault', 
                       'Driver Substance Abuse', 'Driver Distracted By', 
                       'Drivers License State', 'Non-Motorist Substance Abuse', 
                       'Related Non-Motorist']]
df_driver_person.insert(0, 'ID', range(1, 1 + len(df_driver_person)))


# Group 5: Incident Details 
df_incident_details = baseCsv[['Collision Type', 'Weather', 'Surface Condition', 
                          'Light', 'Traffic Control', 'Injury Severity', 'Circumstance']]
df_incident_details.insert(0, 'ID', range(1, 1 + len(df_incident_details)))


In [7]:
df_report_case.to_csv('./loaded/report_case.csv', index=False)
df_road_location.to_csv('./loaded/road_location.csv', index=False)
df_vehicle.to_csv('./loaded/vehicle.csv', index=False)
df_driver_person.to_csv('./loaded/driver_person.txt', sep=',', index=False, header=True)
df_incident_details.to_csv('./loaded/incident_details.txt', sep=',', index=False, header=True)